# Lecture 4 - Character Encodings

Tránh lỗi UnicoodeDecodeErrors khi tải tệp CSV.

Trong sổ tay này, chúng ta sẽ làm việc với các mã hóa ký tự khác nhau.

Hãy bắt đầu thôi!

## Thiết lập môi trường của chúng ta

Điều đầu tiên chúng ta cần làm là tải các thư viện mà chúng ta sẽ sử dụng. Tuy nhiên, không phải là bộ dữ liệu của chúng ta, chúng ta sẽ làm điều đó sau!


In [20]:
# Các mô-đun chúng ta sẽ sử dụng
import pandas as pd
import numpy as np

# Mô-đun mã hóa ký tự hữu ích
import charset_normalizer

# Thiết lập hạt giống để tái lập kết quả
np.random.seed(0)

## Mã hóa là gì?

**Mã hóa ký tự** là các bộ quy tắc cụ thể để ánh xạ từ các chuỗi byte nhị phân thô (trông như thế này: 0110100001101001) đến các ký tự tạo nên văn bản có thể đọc được bởi con người (như "hi"). Có rất nhiều mã hóa khác nhau, và nếu bạn cố gắng đọc văn bản với mã hóa khác so với mã hóa ban đầu mà nó được viết, bạn sẽ gặp phải văn bản bị rối gọi là "mojibake" (đọc là mo-gee-bah-kay). Đây là một ví dụ về mojibake:

æ–‡å—åŒ–ã??

Bạn cũng có thể gặp phải các ký tự "không xác định". Đó là những gì hiển thị khi không có sự ánh xạ giữa một byte cụ thể và một ký tự trong mã hóa bạn đang sử dụng để đọc chuỗi byte, và chúng trông như thế này:

����������

Các lỗi không khớp mã hóa ký tự ít phổ biến hơn ngày nay so với trước đây, nhưng nó vẫn là một vấn đề. Có rất nhiều mã hóa ký tự khác nhau, nhưng mã hóa chính mà bạn cần biết là UTF-8.

> UTF-8 là **mã hóa văn bản** chuẩn. Tất cả mã Python đều ở định dạng UTF-8 và, lý tưởng nhất, tất cả dữ liệu của bạn cũng nên như vậy. Khi dữ liệu không phải là UTF-8, đó là khi bạn gặp phải vấn đề.

Trước đây, việc xử lý mã hóa trong Python 2 khá khó khăn, nhưng may mắn là trong Python 3, nó đơn giản hơn rất nhiều. (Kaggle Notebooks chỉ sử dụng Python 3.) Có hai kiểu dữ liệu chính bạn sẽ gặp khi làm việc với văn bản trong Python 3. Một là chuỗi, đây là kiểu dữ liệu văn bản mặc định.

In [21]:
# Bắt đầu với một chuỗi
before = "This is the euro symbol: €"

# Kiểm tra xem kiểu dữ liệu của nó là gì
type(before)

str

Dữ liệu còn lại là kiểu dữ liệu [bytes](https://docs.python.org/3.1/library/functions.html#bytes), đây là một chuỗi các số nguyên. Bạn có thể chuyển đổi một chuỗi thành bytes bằng cách chỉ định mã hóa mà nó đang sử dụng:

In [22]:
# Mã hóa nó sang một mã hóa khác, thay thế các ký tự gây ra lỗi
after = before.encode("utf-8", errors="replace")

# Kiểm tra kiểu dữ liệu
type(after)

bytes

Nếu bạn nhìn vào một đối tượng bytes, bạn sẽ thấy nó có một chữ b phía trước, sau đó có thể là một số văn bản. Điều này là vì bytes được in ra như thể chúng là các ký tự được mã hóa trong ASCII. (ASCII là một mã hóa ký tự cũ không thực sự phù hợp để viết bất kỳ ngôn ngữ nào ngoài tiếng Anh.) Ở đây, bạn có thể thấy rằng ký hiệu euro của chúng ta đã bị thay thế bằng một số mojibake trông như "\xe2\x82\xac" khi nó được in ra như thể đó là một chuỗi ASCII.

In [23]:
# Hãy xem các byte trông như thế nào
after

b'This is the euro symbol: \xe2\x82\xac'

Khi chúng ta chuyển đổi lại bytes thành chuỗi với mã hóa chính xác, chúng ta có thể thấy rằng văn bản của chúng ta đã được hiển thị đúng, điều này thật tuyệt! :)

In [24]:
# Chuyển đổi nó trở lại utf-8
print(after.decode("utf-8"))

This is the euro symbol: €


However, when we try to use a different encoding to map our bytes into a string, we get an error. This is because the encoding we're trying to use doesn't know what to do with the bytes we're trying to pass it. You need to tell Python the encoding that the byte string is actually supposed to be in.

> You can think of different encodings as different ways of recording music. You can record the same music on a CD, cassette tape or 8-track. While the music may sound more-or-less the same, you need to use the right equipment to play the music from each recording format. The correct decoder is like a cassette player or a CD player. If you try to play a cassette in a CD player, it just won't work. 

In [25]:
# Cố gắng giải mã các byte của chúng ta với mã hóa ascii
print(after.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 25: ordinal not in range(128)

Chúng ta cũng có thể gặp vấn đề nếu cố gắng sử dụng mã hóa sai để ánh xạ từ chuỗi sang bytes. Như tôi đã nói trước đây, chuỗi trong Python 3 mặc định là UTF-8, vì vậy nếu cố gắng xử lý chúng như thể chúng thuộc mã hóa khác, chúng ta sẽ gặp phải vấn đề.

Ví dụ, nếu chúng ta cố gắng chuyển đổi một chuỗi thành bytes theo mã hóa ASCII bằng cách sử dụng `encode()`, chúng ta có thể yêu cầu các byte giống như khi văn bản được mã hóa bằng ASCII. Tuy nhiên, vì văn bản của chúng ta không phải là ASCII, sẽ có một số ký tự mà ASCII không thể xử lý. Chúng ta có thể tự động thay thế những ký tự mà ASCII không thể xử lý. Tuy nhiên, nếu làm vậy, bất kỳ ký tự nào không thuộc ASCII sẽ chỉ được thay thế bằng ký tự không xác định. Sau đó, khi chúng ta chuyển đổi các byte lại thành chuỗi, ký tự đó sẽ bị thay thế bằng ký tự không xác định. Phần nguy hiểm ở đây là không có cách nào để xác định ký tự đó *lẽ ra* phải là gì. Điều này có nghĩa là chúng ta có thể đã làm cho dữ liệu của mình trở nên vô dụng!

In [26]:
# Bắt đầu với một chuỗi
before = "This is the euro symbol: €"

# Mã hóa nó sang một mã hóa khác, thay thế các ký tự gây ra lỗi
after = before.encode("ascii", errors = "replace")

# Chuyển nó lại thành utf-8
print(after.decode("ascii"))

# Chúng ta đã mất chuỗi byte gốc ban đầu! Nó đã bị 
# Thay thế bằng chuỗi byte gốc cho ký tự không xác định :(

This is the euro symbol: ?


Điều này là xấu và chúng ta muốn tránh làm như vậy! Tốt hơn nhiều nếu chuyển tất cả văn bản của chúng ta sang UTF-8 ngay khi có thể và giữ nó trong mã hóa đó. Thời điểm tốt nhất để chuyển đổi đầu vào không phải UTF-8 thành UTF-8 là khi bạn đọc các tệp, điều này chúng ta sẽ thảo luận trong phần tiếp theo.

## Đọc tệp với vấn đề mã hóa

Hầu hết các tệp mà bạn sẽ gặp có thể được mã hóa với UTF-8. Đây là mã hóa mà Python mong đợi mặc định, vì vậy hầu hết thời gian bạn sẽ không gặp phải vấn đề gì. Tuy nhiên, đôi khi bạn sẽ gặp phải lỗi như thế này:

In [27]:
# Cố gắng đọc một tệp không phải UTF-8
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 7955: invalid start byte

Lưu ý rằng chúng ta nhận được cùng một lỗi `UnicodeDecodeError` mà chúng ta gặp phải khi cố gắng giải mã các byte UTF-8 như thể chúng là ASCII! Điều này cho chúng ta biết rằng tệp này thực sự không phải là UTF-8. Tuy nhiên, chúng ta không biết mã hóa thực tế của nó là gì. Một cách để tìm ra là thử và kiểm tra một loạt các mã hóa ký tự khác nhau và xem có mã hóa nào phù hợp không. Tuy nhiên, một cách tốt hơn là sử dụng mô-đun `charset_normalizer` để thử và tự động đoán mã hóa chính xác là gì. Điều này không đảm bảo 100% chính xác, nhưng thường nhanh hơn nhiều so với việc chỉ thử đoán.

Tôi sẽ chỉ xem xét mười nghìn byte đầu tiên của tệp này. Thông thường, đây là đủ để có thể đoán chính xác mã hóa và nhanh hơn nhiều so với việc cố gắng kiểm tra toàn bộ tệp. (Đặc biệt là với một tệp lớn, việc này có thể rất chậm.) Một lý do khác để chỉ xem xét phần đầu của tệp là vì chúng ta có thể thấy qua thông báo lỗi rằng vấn đề đầu tiên là ký tự thứ 11. Vì vậy, chúng ta có thể chỉ cần nhìn vào phần đầu của tệp để tìm ra vấn đề đang xảy ra.

In [28]:
# Xem xét mười nghìn byte đầu tiên để đoán mã hóa ký tự
with open("ks-projects-201801.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(10000))

# Kiểm tra xem mã hóa ký tự có thể là gì
print(result)

{'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


Vậy là `charset_normalizer` có độ tin cậy 73% rằng mã hóa đúng là "Windows-1252". Hãy cùng kiểm tra xem điều đó có đúng không:

In [29]:
# Đọc tệp với mã hóa được phát hiện bởi charset_normalizer
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv", encoding='Windows-1252')

# Xem vài dòng đầu tiên
kickstarter_2016.head()

C:\Users\minh PC\AppData\Local\Temp\ipykernel_12372\900182071.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  kickstarter_2016 = pd.read_csv("ks-projects-201612.csv", encoding='Windows-1252')


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


Đúng rồi, có vẻ như `charset_normalizer` đã đúng! Tệp đã được đọc mà không gặp vấn đề gì (mặc dù chúng ta nhận được cảnh báo về kiểu dữ liệu) và khi xem qua vài dòng đầu tiên, nó có vẻ ổn.

> **Nếu mã hóa mà charset_normalizer đoán không đúng thì sao?** Vì `charset_normalizer` cơ bản chỉ là một công cụ đoán, đôi khi nó sẽ đoán sai mã hóa. Một điều bạn có thể thử là xem xét nhiều hoặc ít phần của tệp và xem có kết quả khác nhau không, rồi thử lại với kết quả đó.

## Lưu tệp của bạn với mã hóa UTF-8

Cuối cùng, sau khi đã trải qua tất cả các bước để đưa tệp của bạn về UTF-8, bạn có thể muốn giữ nó ở mã hóa đó. Cách dễ nhất để làm điều đó là lưu tệp của bạn với mã hóa UTF-8. Tin vui là, vì UTF-8 là mã hóa chuẩn trong Python, khi bạn lưu tệp, nó sẽ được lưu với mã hóa UTF-8 mặc định:

In [30]:
# Lưu tệp của bạn (mặc định sẽ được lưu dưới dạng UTF-8!)
kickstarter_2016.to_csv("ks-projects-201801-utf8.csv")

Pretty easy, huh? :)

## Your turn! 

[**Deepen your understanding**](https://www.kaggle.com/kernels/fork/10824401) with a dataset of fatal police shootings in the US. 

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*